In [30]:
from gensim.models.coherencemodel import CoherenceModel
import math
from gensim.corpora.dictionary import Dictionary
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from src.utility import load_samples_with_numpy

df = load_samples_with_numpy(
    "/Users/robinfeldmann/TopicAnalysisRUWTweets/Data/Embeddings/case_study"
)

In [12]:
df["nouns_adjs_verbs"] = df["nouns"] + df["adjs_verbs"]
df["nouns_adjs_verbs_translated"] = df["translated_nouns"] + df["translated_adjs_verbs"]

In [4]:
df.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,translated,tweetcreatedts,tweetid,week,cleaned_text_translated,cleaned_text,lemmas,adjs_verbs,nouns,entities,emojis,translated_lemmas,translated_adjs_verbs,translated_nouns,translated_entities,translated_emojis,lang,topics,Unnamed: 21,topics_robin,kmeans_cleaned_text_topics,kmeans_lemmas_topics,nouns_topics,embeddings_lemmas,cleaned_text_embeddings,embeddings_nouns
0,0,124,124,124,#UkraineKrieg: Die #EU-#Sanktionen gegen #Russ...,#UkraineWar: The #EU #sanctions against #Russi...,2022-02-28 11:13:37,1498255060215767040,2022-09,ukrainewar: the eu sanctions against russia...,ukrainekrieg: die eusanktionen gegen russlan...,"[Ukrainekrieg, eusanktionen, gegen, Russland, ...",[],"[Ukrainekrieg, eusanktionen, Russland, Überbli...",[russland/LOC],[],"[ukrainewar, eu, sanction, against, russia, at...","[ukrainewar, ukrainerussiawar]","[eu, sanction, russia, glance, ukrainerussiacr...",[russia/GPE],[],de,sanctions,NaN,NaN,1,1,2,"[-0.009074805, -0.0574717, -0.030188987, -0.06...","[0.015300176, -0.043482374, -0.01747574, -0.05...","[-0.028477553, 0.02194917, -0.052988503, -0.05..."
1,1,136,136,136,"Vogt-Wuchter: ""Wir sind schockiert über den Kr...","Vogt-Wuchter: ""We are shocked by the war in Uk...",2022-03-01 09:00:11,1498583868449776128,2022-09,"vogt-wuchter: ""we are shocked by the war in uk...","vogt-wuchter: ""wir sind schockiert über den kr...","[vogt-wuchter, wir, schockiert, über, Krieg, i...","[anzetteln, fliehen, ukrain]","[vogt-wuchter, Krieg, Ukraine, Europa, Russlan...","[vogt-wuchter/LOC, ukraine/LOC, europa/LOC, ru...",[],"[vogt, wuchter, we, shock, by, war, in, ukrain...","[vogt, shock, launch, international, flee, ukr...","[wuchter, war, ukraine, middle, europe, russia...","[europe/LOC, russia/GPE]",[],de,unjustified_war,NaN,NaN,1,0,0,"[0.024267722, -0.016830716, -0.0496836, -0.048...","[0.06050161, -0.034141194, -0.026284462, -0.05...","[-0.014658581, -0.03167732, -0.04351097, -0.06..."
2,3,156,156,156,Das Innenministerium der #Ukraine meldete am S...,The Interior Ministry of #Ukraine reported on ...,2022-02-28 07:42:15,1498201871118475008,2022-09,the interior ministry of ukraine reported on ...,das innenministerium der ukraine meldete am s...,"[Innenministerium, Ukraine, melden, an, sonnta...","[melden, getötet, verwundet, zählen]","[Innenministerium, Ukraine, Zivilist, Angriff,...","[russlands/LOC, ukraine/LOC]",[],"[interior, ministry, of, ukraine, report, on, ...","[report, kill, include, further, wound, count,...","[interior, ministry, ukraine, sunday, evening,...","[sunday/DATE, 352/CARDINAL, russia/GPE, 14/CAR...",[],de,people_killed,NaN,NaN,2,0,0,"[-0.00031765725, -0.036075648, -0.040189527, -...","[0.0012358468, -0.01779583, 0.032502245, -0.04...","[-0.046677843, 0.0270702, -0.058521565, -0.065..."
3,4,212,212,212,@Bundeskanzler Einfuhren aus Russland sofort S...,@Federal Chancellor STOP imports from Russia i...,2022-03-11 16:52:55,1502326713505751040,2022-10,chancellor stop imports from russia immediatel...,"einfuhren aus russland sofort stoppen, ob gas ...","[einfuhren, aus, Russland, sofort, stoppen, Ga...","[einfuhren, stoppen, Kohle, Kriegsverbrechen, ...","[Russland, Gas, Deutschland, mitschuldig, Stop...","[russland/LOC, deutschland/LOC]",[],"[chancellor, stop, import, from, russia, immed...","[stop, complicit, stopputinnow]","[chancellor, import, russia, gas, coal, case, ...","[russia/GPE, germany/GPE]",[],de,sanctions,NaN,NaN,0,3,2,"[-0.0025235536, -0.023199191, -0.05079851, -0....","[0.016103381, -0.00022359387, -0.055152107, -0...","[-0.044738892, 0.017367825, -0.049746756, -0.0..."
4,5,213,213,213,#Ukraine\nAlle schimpfen über hohen benzinprei...,#Ukraine\nEveryone complains about high gas pr...,2022-03-13 09:58:03,1502947084948984064,2022-10,ukraine everyone complains about high gas pric...,ukraine alle schimpfen über hohen benzinpreise...,"[ukrainen, Schimpfe, über, hoch, benzinpreise,...","[ukrainen, hoch, finanzieren, kurz, finanzieren]",

In [20]:
class DiversityScore:

    def __init__(self, top_n: int = 40):
        self.top_n = top_n

    def score(self, topics: list[list[str]]):
        unique_words = set()
        for topic in topics:
            unique_words = unique_words.union(set(topic[: self.top_n]))
        td = len(unique_words) / (self.top_n * len(topics))

        return td

In [21]:
diversity_score = DiversityScore()

In [10]:
import pandas as pd


def prepare_coherence_space(
    df: pd.DataFrame, topics: list[int], evaluation_on: str
) -> tuple[list, list]:

    def create_lists(df, on_col="nouns"):

        k = []
        df[on_col].apply(k.extend)

        return k

    df_clustered = pd.DataFrame()
    df_clustered["topics"] = topics

    df_clustered[evaluation_on] = df[evaluation_on]

    list_of_topics: list[list[str]] = (
        df_clustered.groupby("topics")
        .apply(lambda x: create_lists(x, evaluation_on))
        .to_list()
    )
    list_of_texts: list[list[str]] = df_clustered[evaluation_on]

    return list_of_topics, list_of_texts

In [23]:
list_of_topics, _ = prepare_coherence_space(
    df, df["kmeans_lemmas_topics"], "translated_lemmas"
)

In [24]:
diversity_score.score(list_of_topics)

0.725